In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [22]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [23]:
result = model(tokens)

In [24]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [25]:
int(torch.argmax(result.logits))+1

4

In [39]:
r = requests.get('https://www.yelp.com/biz/four-chairs-san-francisco?osq=Restaurants')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [40]:
reviews

["Great eggs benedict best I've had so far, freshly made orange juice not the carton stuff.",
 "My friend randomly picked this place and I must tell you, THIS IS THEEEE SPOT TO BE for breakfast and brunch. We were a bit shocked about the line on a Sunday noon but a quick glance inside explained everything. Luckily the wait was only 15-20min. This place is packed like crazy. The food is super good. Didn't get to try any (alcoholic) drinks but I'm sure those are fabulous as well.Staff is friendly and quick to serve you.",
 'Coffee volcano, YES please! A few things I love about this place:1. Potatoes are crisp, not soggy and oily 2. Portions are great. I hope they never skimp3. Coffee and matcha choices are deelishMy go to: Mascarpone pancakesLox omelette Coffee volcano',
 "This restaurant is nestled in the mission and I stumbled across it on a random morning!Parking around this area is good, if you can't find anything on the street, there is a Safeway to park in need be. the restaurant d

In [41]:
import numpy as np
import pandas as pd


In [42]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [43]:
df['review'].iloc[0]


"Great eggs benedict best I've had so far, freshly made orange juice not the carton stuff."

In [44]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [45]:
sentiment_score(df['review'].iloc[1])

5

In [46]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [47]:
df

,review,sentiment
0,"Great eggs benedict best I've had so far, fres...",5
1,My friend randomly picked this place and I mus...,5
2,"Coffee volcano, YES please! A few things I lov...",4
3,This restaurant is nestled in the mission and ...,4
4,Four Chairs was our final stop on our San Fran...,5
5,Mascarpone pancakes - 3 super fluffy pancakes ...,4
6,We decided to try Four Chairs after seeing all...,4
7,Not a bad spot but it does get busy. I came he...,3
8,Just an update warning -- no espresso drinks h...,3
9,There are plenty of pictures of the incredibly...,2


In [48]:
df['review'].iloc[3]

"This restaurant is nestled in the mission and I stumbled across it on a random morning!Parking around this area is good, if you can't find anything on the street, there is a Safeway to park in need be. the restaurant definitely has a cozy feel to it. the service was fast and attentive food - 4/51. Spicy Korean Fried Eggs Benedict - this was bomb! the chicken was well marinated and it tied in well with the eggs and potatoes 2. Creme Brûlée French Toast - if you have a sweet tooth, this is the one for you. I loved the caramelized sugar on top to give it that creme brûlée traditional feel. The toast itself was fluffy and well seasoned. 3. Pancakes - they give you a good amount it's also on the sweeter side. I would just say this was average."